In [2]:
import polars as pl
df = pl.read_csv(r"D:\Users\shap1\Desktop\DE_HSE_REPOS\hse_de_pyintro_2024\HW6\Electronic_sales_Sep2023-Sep2024.csv")

df.describe()

statistic,Customer ID,Age,Gender,Loyalty Member,Product Type,SKU,Rating,Order Status,Payment Method,Total Price,Unit Price,Quantity,Purchase Date,Shipping Type,Add-ons Purchased,Add-on Total
str,f64,f64,str,str,str,str,f64,str,str,f64,f64,f64,str,str,str,f64
"""count""",20000.0,20000.0,"""20000""","""20000""","""20000""","""20000""",20000.0,"""20000""","""20000""",20000.0,20000.0,20000.0,"""20000""","""20000""","""15132""",20000.0
"""null_count""",0.0,0.0,"""0""","""0""","""0""","""0""",0.0,"""0""","""0""",0.0,0.0,0.0,"""0""","""0""","""4868""",0.0
"""mean""",10483.52655,48.9941,null,null,null,null,3.09395,null,null,3180.133419,578.631867,5.48555,null,null,null,62.244848
"""std""",5631.732525,18.038745,null,null,null,null,1.223764,null,null,2544.978675,312.274076,2.870854,null,null,null,58.058431
"""min""",1000.0,18.0,"""#N/A""","""No""","""Headphones""","""HDP456""",1.0,"""Cancelled""","""Bank Transfer""",20.75,20.75,1.0,"""2023-09-24""","""Expedited""","""Accessory""",0.0
"""25%""",5478.0,33.0,null,null,null,null,2.0,null,null,1139.68,361.18,3.0,null,null,null,7.62
"""50%""",11000.0,49.0,null,null,null,null,3.0,null,null,2534.49,463.96,5.0,null,null,null,51.7
"""75%""",15504.0,65.0,null,null,null,null,4.0,null,null,4639.6,791.19,8.0,null,null,null,93.84
"""max""",19998.0,80.0,"""Male""","""Yes""","""Tablet""","""TBL345""",5.0,"""Completed""","""Paypal""",11396.8,1139.68,10.0,"""2024-09-23""","""Standard""","""Impulse Item,Impulse Item,Impu…",292.77


In [15]:
pay_pref_df = df.filter(pl.col("Order Status") == 'Completed') \
    .group_by(["Customer ID", "Payment Method"]).len(name='num_trans') \
    .with_columns(rn = pl.col("num_trans").cum_count().over(partition_by=["Customer ID"])) \
    .filter(pl.col('rn') == 1).select(["Customer ID", "Payment Method"])

result_df = df.filter(pl.col("Order Status") == 'Completed') \
    .group_by("Customer ID").agg(pl.col("Add-on Total").sum(), pl.col("Total Price").sum()) \
    .join(pay_pref_df, on='Customer ID').rename({
        'Customer ID': 'cus_id',
        'Add-on Total': 'total_addon_spent',
        'Total Price': 'total_spent',
        'Payment Method': 'preferred_payment_method'
    })

 
result_df.sort("cus_id").head(10)

cus_id,total_addon_spent,total_spent,preferred_payment_method
i64,f64,f64,str
1000,26.09,741.09,"""Paypal"""
1002,60.16,5020.6,"""Cash"""
1003,35.56,41.5,"""Cash"""
1004,65.78,83.0,"""Credit Card"""
1005,75.33,11779.11,"""Debit Card"""
1006,90.38,6645.94,"""Paypal"""
1008,65.85,3379.32,"""Cash"""
1011,70.17,7911.9,"""Credit Card"""
1014,8.3,4747.14,"""Cash"""
